## Prepare the Dataset

In [2]:
# download the dataset
!gdown https://drive.google.com/u/0/uc?id=11Zqe8K16CkCXDPFPLc5M7OqM8gX3ZuVa&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=11Zqe8K16CkCXDPFPLc5M7OqM8gX3ZuVa
To: /content/DatasetBangla.zip
100% 10.5M/10.5M [00:00<00:00, 159MB/s]


In [ ]:
# extract the dataset
!unzip '/content/DatasetBangla.zip'

In [26]:
import pandas as pd
import glob

paths = glob.glob('/content/DatasetBangla/*/*/*.*')

df = pd.DataFrame({'Path': paths})

def extract_label(path):
    return path.split('/')[-2]

def extract_set(path):
    return path.split('/')[-3]

df['Label'] = df['Path'].apply(extract_label)
df = pd.concat([df, pd.get_dummies(df['Label'])], axis = 1)

df['Set'] = df['Path'].apply(extract_set)

train_df = df.loc[df['Set'] == 'Train'].iloc[:, :-1]
test_df = df.loc[df['Set'] == 'Test'].iloc[:, :-1]

In [27]:
train_df.sample(2)

,Path,Label,172,173,174,175,176,177,178,179,...,212,213,214,215,216,217,218,219,220,221
3275,/content/DatasetBangla/Train/179/bcc000167.bmp,179,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5018,/content/DatasetBangla/Train/192/bcc000037.bmp,192,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
test_df.sample(2)

,Path,Label,172,173,174,175,176,177,178,179,...,212,213,214,215,216,217,218,219,220,221
2800,/content/DatasetBangla/Test/183/bcc000054.bmp,183,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,/content/DatasetBangla/Test/204/bcc000042.bmp,204,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Custom Dataloader

In [22]:
import torch
import torchvision
import cv2

In [29]:
class CustomDatasetLoader(torch.utils.data.Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        path = self.df.iloc[idx, 0]
        img = cv2.imread(path)
        
        if self.transform:
            img = self.transform(img)
        
        label = torch.from_numpy(self.df.iloc[idx, 2:].values.astype(float))

        return (img, label)

In [30]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.Grayscale(num_output_channels=3),
    torchvision.transforms.ToTensor(),
])

In [38]:
train_data = CustomDatasetLoader(df = train_df, transform = transform)
test_data = CustomDatasetLoader(df = test_df, transform = transform)

train_dataloader = torch.utils.data.DataLoader(
    dataset = train_data, batch_size = 32, shuffle = True, num_workers = 1, drop_last = True
)
test_dataloader = torch.utils.data.DataLoader(
    dataset = test_data, batch_size = 64, shuffle = True, num_workers = 1, drop_last = False
)

In [42]:
for idx, (images, labels) in enumerate(train_dataloader):
    print(images.shape, labels.shape)
    if idx >= 2:
        break

torch.Size([32, 3, 128, 128]) torch.Size([32, 50])
torch.Size([32, 3, 128, 128]) torch.Size([32, 50])
torch.Size([32, 3, 128, 128]) torch.Size([32, 50])


In [44]:
for idx, (images, labels) in enumerate(test_dataloader):
    print(images.shape, labels.shape)
    if idx >= 2:
        break

torch.Size([64, 3, 128, 128]) torch.Size([64, 50])
torch.Size([64, 3, 128, 128]) torch.Size([64, 50])
torch.Size([64, 3, 128, 128]) torch.Size([64, 50])
